* *pn* - Peter Norvig's algorithm [#norvig_spellchecker]_ (default)
* *phunspell* - A spell checker utilizing spylls, a port of Hunspell.
* *symspellpy* - symspellpy is a Python port of SymSpell v6.5.
* *wanchanberta_thai_grammarly* - WanchanBERTa Thai Grammarly

In [16]:
# -*- coding: utf-8 -*-
from pythainlp.spell import correct
from pythainlp.tokenize import word_tokenize

# ระบุ engine ที่จะใช้เพื่อความแม่นยำ
ENGINE = 'phunspell'

print(f"--- แก้ไขคำผิดด้วย Engine: {ENGINE} ---")

incorrect_words = [
    "บริการที่ปรึกษาคำแถวรดลาดศิริทัล",
    "คาบำรุงรักษาระบบรายปี",
    "คาเขา Cloud Server - Business Package",
    "พัฒนาและออกแบบเว็บไซต์ (E-commerce)",
    "เรียนโขการข้าระเงิน: ข้าระพันทิเมื่อโต่วันเอกสาร ข้อมูลการโอนเงิน", 
]

for word in incorrect_words:
    corrected_word = correct(word, engine=ENGINE)
    print(f"คำผิด: '{word}' -> แก้เป็น: '{corrected_word}'")

--- แก้ไขคำผิดด้วย Engine: phunspell ---


IndexError: list index out of range

In [17]:
import re
from pythainlp.ner import NER

# The text from the invoice
text = """
# บริษัท โกลบอล โลจิสติกส์ โซลูชั่นส์

999/9 อาคารคี ออฟฟิศเคส แอท เซ็นทรัลเวิลด์ ชั้น 40 ถ.พระราม 1 แขวงปทุมวัน เขตปทุมวัน กรุงเทพฯ 10330 โทร: 02-123-4567 เลขประจำตัวผู้เสียภาษี: 0879189666478

ลูกค้า: หจก. สยามการพิมพ์ ที่อยู่: 101 อาคารทรู ศิริทัล พารค ชั้น 7 ถ.สุขุมวิท แขวงบางจาก เขตพระพุทธ กรุงเทพฯ 103984/2025

เรียน: คุณสุนีย์ โทร: 02-123-4567 เลขประจำตัวผู้เสียภาษี: 0560437188559

|  ลำดับ รายการ | จำนวน | ราคา/หน่วย | จำนวนเงิน  |
| --- | --- | --- | --- |
|  1 บริการที่ปรึกษาคำแถวรดลาดศิริทัล (เดือน มี.ย.) | 1 | 30,000.00 | 30,000.00  |
|  2 คาบำรุงรักษาระบบรายปี | 1 | 20,000.00 | 20,000.00  |
|  3 คาเขา Cloud Server - Business Package | 3 | 2,500.00 | 7,500.00  |
|  4 พัฒนาและออกแบบเว็บไซต์ (E-commerce) | 1 | 75,000.00 | 75,000.00  |
|  เรียนโขการข้าระเงิน: ข้าระพันทิเมื่อโต่วันเอกสาร ข้อมูลการโอนเงิน: ธ.กติกริไทย บัญชีออมทรัพย์ 012-3-45678-9 | ยอดรวม (Subtotal) | 132,500.00 |   |
|   | ภาษีมูลค่าเพิ่ม (VAT 7%) | 9,275.00 |   |
|   | ยอดข้าระทั้งสิ้น | 141,775.00 |   |

ผู้อนุมัติ

ผู้รับเงิน / ผู้ลงของ
"""

print("---  extracting key information using regular expressions ---")

# --- Function to extract specific patterns ---
def find_pattern(pattern, text):
    match = re.search(pattern, text)
    return match.group(1).strip() if match else "Not Found"

# --- Define regex patterns ---
supplier_name_pattern = r"#\s*(บริษัท.*?โซลูชั่นส์)"
supplier_tax_id_pattern = r"เลขประจำตัวผู้เสียภาษี:\s*(\d+)"
customer_name_pattern = r"ลูกค้า:\s*(หจก\.\s*.*?การพิมพ์)"
customer_attn_pattern = r"เรียน:\s*(คุณ\w+)"
customer_tax_id_pattern = r"ผู้เสียภาษี:\s*(\d+)"
subtotal_pattern = r"ยอดรวม\s*\(Subtotal\)\s*\|\s*([\d,]+\.\d{2})"
vat_pattern = r"ภาษีมูลค่าเพิ่ม\s*\(VAT 7%\)\s*\|\s*([\d,]+\.\d{2})"
total_pattern = r"ยอดข้าระทั้งสิ้น\s*\|\s*([\d,]+\.\d{2})"

# --- Extract and print information ---
print(f"Supplier Name: {find_pattern(supplier_name_pattern, text)}")
print(f"Supplier Tax ID: {find_pattern(supplier_tax_id_pattern, text)}")
print(f"Customer Name: {find_pattern(customer_name_pattern, text)}")
print(f"Attention To: {find_pattern(customer_attn_pattern, text)}")
print(f"Customer Tax ID: {find_pattern(customer_tax_id_pattern, text)}")
print("-" * 20)
print(f"Subtotal: {find_pattern(subtotal_pattern, text)}")
print(f"VAT (7%): {find_pattern(vat_pattern, text)}")
print(f"Total: {find_pattern(total_pattern, text)}")


print("\n--- extracting entities using pythainlp NER ---")

# --- Use pythainlp for Named Entity Recognition ---
# The 'thainer' model is effective for general-purpose entity recognition.
ner = NER(engine="thainer")
entities = ner.tag(text, pos=False)

# --- Filter and display relevant entities ---
# We look for organizations, persons, and locations.
relevant_entities = {
    "ORGANIZATION": set(),
    "PERSON": set(),
    "LOCATION": set(),
}

for word, tag in entities:
    if "B-" in tag:  # B- indicates the beginning of an entity
        entity_type = tag.replace("B-", "")
        if entity_type in relevant_entities:
            relevant_entities[entity_type].add(word)

for entity_type, items in relevant_entities.items():
    print(f"{entity_type}: {', '.join(items)}")

ModuleNotFoundError: No module named 'pythainlp.ner'